# Synchrony Datathalon 2025

## Load the data

In [4]:
import pandas as pd
import numpy as np

# Load all the datasets
account_df = pd.read_csv('data/account_dim_20250325.csv')
fraud_case_df = pd.read_csv('data/fraud_claim_case_20250325.csv')
fraud_tran_df = pd.read_csv('data/fraud_claim_tran_20250325.csv')
rams_df = pd.read_csv('data/rams_batch_cur_20250325.csv')
statement_df = pd.read_csv('data/statement_fact_20250325.csv')
syfid_df = pd.read_csv('data/syf_id_20250325.csv')
transaction_df = pd.read_csv('data/transaction_fact_20250325.csv')
wrld_transaction_df = pd.read_csv('data/wrld_stor_tran_fact_20250325.csv')


## Pre Process functions

In [5]:
# Function to clean the RAMS dataset
def clean_rams(df):
    # Convert date column to datetime format
    df['cu_processing_date'] = pd.to_datetime(df['cu_processing_date'])

    # Keep only the latest processing date for each account
    latest_df = df.sort_values(by=['cu_processing_date'], ascending=False).drop_duplicates(subset=['cu_account_nbr'], keep='first')

    # drop the ca_cash_bal_pct_crd_line column
    df = df.drop(columns=['ca_cash_bal_pct_crd_line'])
    # irrelevant column as the values are all 0

    #drop the cu_nbr_days_dlq column
    df = df.drop(columns=['cu_nbr_days_dlq'])
    #redundant as theres a simialr column with months which is more useful

    #For all values in column cu_crd_bureau_scr , replace the value 0 with median of the column cu_crd_bureau_scr
    df['cu_crd_bureau_scr'] = df['cu_crd_bureau_scr'].replace(0, df['cu_crd_bureau_scr'].median())

    #drop the column cu_next_crd_line_rev_date
    df = df.drop(columns=['cu_next_crd_line_rev_date'])
    #irrelevant column as majority values are 0

    #dropping columns
    useless_col = [
        'cu_cur_balance',
        'ca_mob',
        'cu_rnd_nbr',
        'rb_crd_gr_new_crd_gr',
        'cu_processing_date',
        'mo_tot_sales_array_1',
        'mo_tot_sales_array_2',
        'mo_tot_sales_array_3',
        'mo_tot_sales_array_4',
        'mo_tot_sales_array_5',
        'mo_tot_sales_array_6'
    ]

    #Droping the above generated columns
    df = df.drop(columns=useless_col)

    # for the values 999999999999999 in cu_cash_line_am replace them with 20% of corresponding valur of cu_crd_bureau_scr column 
    df['cu_cash_line_am'] = df.apply(
        lambda row: row['cu_crd_bureau_scr'] * 0.2 if row['cu_cash_line_am'] == 999999999999999 else row['cu_cash_line_am'],
        axis=1
    )
    # round it off to 2 decimal places
    df['cu_cash_line_am'] = df['cu_cash_line_am'].round(2)

    #drop duplicate rows with duplicate values in the column cu_account_nbr
    df = df.drop_duplicates(subset=['cu_account_nbr'])

    return df

## Apply all the preprocessing

In [6]:
updated_rams_df = clean_rams(rams_df)
updated_rams_df.head()

,cu_bhv_scr,ca_cash_bal_pct_cash_line,cu_nbr_of_plastics,ca_avg_utilz_lst_6_mnths,cu_cash_line_am,cu_crd_bureau_scr,cu_crd_line,cu_cur_nbr_due,ca_current_utilz,cu_account_nbr,cu_line_incr_excl_flag,ca_max_dlq_lst_6_mnths,ca_mnths_since_active,ca_mnths_since_cl_chng,ca_nsf_count_lst_12_months,cu_otb,rb_new_bhv_scr,ca_avg_utilz_lst_3_mnths
0,779,0,2,2,5000.0,782,25000,0,4,37eHiwRArQ0A7jhs,N,0,0,45,0,24002.44,776,2
1,735,999,1,2,0.0,836,1500,0,0,UV2Z1ODsmodPM2eO,N,0,1,15,0,1500.00,735,0
2,745,0,1,2,8000.0,832,40000,0,3,VqweW0TxF93M2EF5,N,0,0,7,0,38982.45,748,1
3,13,0,1,0,60.0,684,300,0,0,pfKs8HDpmFV3b1zL,N,0,12,15,0,300.00,13,0
4,6,999,1,7,0.0,681,5200,0,16,35VRb2UTGapQk6wE,N,0,0,1,0,4368.27,6,7
